In [54]:
# For today's fun we'll be using the following packages:
import requests
from bs4 import BeautifulSoup
import time as time
import pandas as pd
import pickle

# Build the scraper!

In [90]:
def techmeme_scrap(search_page, dataframe):
    """This function parses a url request from the TechMeme search and appends the results to an existing pandas dataframe."""
    for num, item in enumerate(BeautifulSoup(search_page.text, 'lxml').find_all('div', {'class':'item'})):
        try:
            in_context = item.find('div', {'class' : 'iinf'}).a['href']
            for i in range(6):
                context_url = requests.get(in_context)
                if context_url.status_code == 200:
                    break
                else:
                    print('Hand smack!')
                    time(60)
                    continue
        except:
            if num <10:
                print('Item: ', num+1, '\nCannot find "in context" link.')
            continue
        date = item.find('span', {'class' : 'idate'}).text
        original_source = item.find('a').text
        original_title = item.find('div', {'class':'ii'}).text
        extra_sources = []
        extra_titles = []
        for context_item in BeautifulSoup(context_url.text, 'lxml').find_all('div', {'class':'item'}):
            try:
                if context_item.find('span')['pml'] == in_context[-9:]:
                    for extra in context_item.find_all('div', {'class':'di'}):
                        extra_sources.append(extra.find('a').text)
                        extra_titles.append(extra.find_all('a')[1].text)
                else:
                    continue
            except:
                pass
        dataframe.loc[len(dataframe)]=[date, original_source, original_title, extra_sources, extra_titles]

def techmeme_search(start_url, dataframe):
    first_request = requests.get(start_url)
    extra_pages = int(BeautifulSoup(first_request.text, 'lxml').find('h2').text.split(' ')[-1].rstrip(':'))//10
    print('Total pages to scrap: ', extra_pages+1)
    print('Page: 1')
    print(start_url)
    techmeme_scrap(first_request, dataframe)
    for page in range(1, extra_pages+1):
        next_url = start_url[:-1] + str((page*10))
        print('Page: ', page+1)
        print(next_url)
        for i in range(6):
            next_request = requests.get(next_url)
            if next_request.status_code == 200:
                techmeme_scrap(next_request, dataframe)
                break
            else:
                print('Hand smack!')
                time(60)
                continue
                

# Run it!

In [91]:
# Initialize your empty DataFrame.
techmeme_df = pd.DataFrame(columns=['date', 'original_source', 'original_title', 'extra_sources', 'extra_titles'])

In [92]:
# Give it a starting url and run the techmeme_search funtion.
start_url = 'https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=0'
techmeme_search(start_url, techmeme_df)

Total pages to scrap:  11
Page: 1
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=0
Page:  2
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=10
Page:  3
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=20
Page:  4
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=30
Page:  5
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=40
Page:  6
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=50
Page:  7
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=60
Page:  8
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=70
Page:  9
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=80
Page:  10
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=90
Page:  11
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&star

In [93]:
# Check that it is the same size as what you were expecting.
techmeme_df.shape

(106, 5)

In [95]:
# Save it in your prefered format.  Or both.  You do you.
pickle.dump(techmeme_df, open('techmeme_df.pkl', 'wb'))
techmeme_df.to_csv('techmeme.csv')